In [ ]:
import os
import json 

from oddstracker.config import DATA_DIR
from oddstracker.domain.kambi_event import KambiData, KambiEvent
from oddstracker.domain.kambi_provider import PROVIDER, KambiProvider
from oddstracker.service.oddscollector import pull_kdata, store_kdata

In [ ]:
kdata = pull_kdata()

In [ ]:
with open(os.path.join(DATA_DIR, "kdata.json"), "w", encoding="utf-8") as f:
    _test_data = kdata[1]
    out = ""
    for bo in _test_data.betOffers:
        out += bo.model_dump_json() + "\n"
    f.write(out)

In [ ]:
with open(os.path.join(DATA_DIR, "betOffers.json"), "w") as f:
    json.dump(kdata[0].betOffers[0].model_dump(), f, indent=2, default=str)
